In [1]:
import tensorflow as tf
import numpy as np

import Poetry
import cn_utils

%load_ext autoreload
%autoreload 2

In [2]:
poetries = cn_utils.readFile('./poetry')

word_to_idx, idx_to_word = cn_utils.get_idx_and_words(poetries)

all files:
['\xe9\xa9\xac\xe8\xaf\x97.txt', '\xe6\x82\xaf\xe5\x86\x9c.txt', '\xe6\xb8\xb8\xe5\xad\x90\xe5\x90\x9f.txt', '\xe9\xbb\x84\xe9\xb9\xa4\xe6\xa5\xbc.txt', '\xe6\x98\x8e\xe6\x97\xa5\xe6\xad\x8c.txt', '\xe6\x9e\xab\xe6\xa1\xa5\xe5\xa4\x9c\xe6\xb3\x8a.txt', '\xe5\x8f\xa4\xe6\xb5\xaa\xe6\x9c\x88\xe8\xa1\x8c.txt', '\xe8\xb5\x8b\xe5\xbe\x97\xe5\x8f\xa4\xe5\x8e\x9f\xe8\x8d\x89\xe9\x80\x81\xe5\x88\xab.txt', '\xe8\xb6\x8a\xe5\xa5\xb3\xe8\xaf\x8d.txt', '\xe7\xa7\x8b\xe6\xb5\xa6\xe6\xad\x8c.txt', '\xe5\x92\x8f\xe9\xb9\x85.txt', '\xe5\xb1\xb1\xe8\xa1\x8c.txt', '\xe8\x8d\x89.txt', '\xe6\x97\xa9\xe5\x8f\x91\xe7\x99\xbd\xe5\xb8\x9d\xe5\x9f\x8e.txt', '\xe9\xa2\x98\xe9\x83\xbd\xe5\x9f\x8e\xe5\x8d\x97\xe5\xba\x84.txt', '\xe4\xb9\x8c\xe8\xa1\xa3\xe5\xb7\xb7.txt', '\xe6\xb1\x9f\xe7\x95\x94\xe7\x8b\xac\xe6\xad\xa5\xe5\xaf\xbb\xe8\x8a\xb1.txt', '\xe7\x8b\xac\xe5\x9d\x90\xe6\x95\xac\xe4\xba\xad\xe5\xb1\xb1.txt', '\xe6\x9c\x9b\xe5\xba\x90\xe5\xb1\xb1\xe7\x80\x91\xe5\xb8\x83.txt', '\xe6\xb1\xa0\xe4\xb

In [3]:
encoded_poetries = []
for pt in poetries:
    encoded_poetries.append(cn_utils.encodePoetry(pt, word_to_idx))

In [4]:
import poetry_utils
data = {}
data = poetry_utils.generate_data(encoded_poetries, data)

In [5]:
type(encoded_poetries[0].title[0])

int

In [6]:
dataset = Poetry.Poetries(encoded_poetries)

poetry length:  28
poetry length:  28
poetry length:  41
poetry length:  69
poetry length:  29
poetry length:  38
poetry length:  30
poetry length:  33
poetry length:  29
poetry length:  29
poetry length:  28
poetry length:  36
poetry length:  51
poetry length:  39
poetry length:  39
poetry length:  37
poetry length:  40
poetry length:  31
poetry length:  39
poetry length:  28
poetry length:  54
poetry length:  77
poetry length:  30
poetry length:  28
poetry length:  31
poetry length:  30
poetry length:  37
poetry length:  30
poetry length:  34
poetry length:  41
poetry length:  30
poetry length:  52
poetry length:  36
poetry length:  28
poetry length:  29
poetry length:  30
poetry length:  54
poetry length:  28
poetry length:  36
poetry length:  37
poetry length:  54
poetry length:  36
poetry length:  28
poetry length:  29
poetry length:  28
poetry length:  31
poetry length:  28
poetry length:  58
poetry length:  36
poetry length:  30


In [7]:
dataset.data_set

array([[  3,   4,   1, ...,   7,   8,   9],
       [  4,   1,   5, ...,   8,   9,  10],
       [  1,   5,   6, ...,   9,  10,  11],
       ..., 
       [263, 263, 215, ..., 241,  63,  86],
       [263, 215,  10, ...,  63,  86,  63],
       [215,  10, 240, ...,  86,  63,  16]], dtype=int32)

In [8]:
dataset[0:1][0].shape

(1, 8)

# 建立tensorflow图


In [9]:
tf.reset_default_graph()
batch_size = 8
vocab_size = len(idx_to_word)
hidden_size = 200
num_steps = dataset.sample_len
lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
#cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * 2)
cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(hidden_size) for _ in range(2)])

input_data = tf.placeholder(tf.int32, [None, None])#第一个维度是batch_size,第二个维度是num_steps
targets = tf.placeholder(tf.int32, [None, None])#跟上面一致

embedding = tf.get_variable(name='embedding', shape=[len(idx_to_word), hidden_size])

inputs = tf.nn.embedding_lookup(embedding, input_data)
outputs = []

initial_state = cell.zero_state(batch_size, tf.float32)

state = initial_state

#with tf.variable_scope('RNN'):
#    for time_step in range(dataset.sample_len):
#        if time_step > 0: tf.get_variable_scope().reuse_variables()
#        cell_output, state = cell(inputs[:,time_step,:], state)
#        outputs.append(cell_output)
        
        
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
#output = tf.reshape(tf.concat(outputs, 1), [-1, hidden_size])
output = tf.reshape(outputs, [-1, hidden_size])


weight = tf.get_variable('weight', [hidden_size, vocab_size])
bias = tf.get_variable('bias', [vocab_size])
logits = tf.matmul(output, weight) + bias

loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(targets, [-1])],
                                                          [tf.ones([batch_size * num_steps], dtype = tf.float32)])
cost = tf.reduce_sum(loss) / batch_size

#final_state = state

trainable_variables = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, trainable_variables), clip_norm=5)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0)
train_op = optimizer.apply_gradients(zip(grads, trainable_variables))

In [11]:
writer = tf.summary.FileWriter('./logs/', tf.get_default_graph())
writer.close()

# 训练

In [12]:
sess = tf.InteractiveSession()


_state = sess.run(initial_state)

sess.run(tf.global_variables_initializer())

In [13]:
T = 5000
predict = None

for i in xrange(T):

    idx = np.random.randint(low = 0, high = dataset.data_set.shape[0], size = (batch_size))
    _cost, _state, _ = sess.run([cost,final_state, train_op], 
                              feed_dict = {input_data: dataset.data_set[idx,:].reshape(batch_size,-1),
                                           initial_state : _state,
                                           targets: dataset.target[idx,:].reshape(batch_size,-1)
                                           })
    if i % 100 == 0:
        print 'The %dth iteration...\n' % i, 'the cost is: ', _cost




The 0th iteration...
the cost is:  51.5848
The 100th iteration...
the cost is:  45.0516
The 200th iteration...
the cost is:  42.1483
The 300th iteration...
the cost is:  40.6873
The 400th iteration...
the cost is:  36.5919
The 500th iteration...
the cost is:  32.2843
The 600th iteration...
the cost is:  26.5092
The 700th iteration...
the cost is:  19.9011
The 800th iteration...
the cost is:  13.2616
The 900th iteration...
the cost is:  13.6391
The 1000th iteration...
the cost is:  13.6563
The 1100th iteration...
the cost is:  13.8703
The 1200th iteration...
the cost is:  10.9874
The 1300th iteration...
the cost is:  7.03209
The 1400th iteration...
the cost is:  11.494
The 1500th iteration...
the cost is:  8.71778
The 1600th iteration...
the cost is:  9.50494
The 1700th iteration...
the cost is:  6.70989
The 1800th iteration...
the cost is:  6.89084
The 1900th iteration...
the cost is:  8.06657
The 2000th iteration...
the cost is:  8.01195
The 2100th iteration...
the cost is:  9.61723
T

# 预测测试，可以看出有那么点样子了。
目前还是只能预测一句话，而且格式得中规中矩，每句都得5个字，而且逗号句号都得标出来。还没想好怎么生成完整的古诗

In [17]:
test_data = np.array(cn_utils.encodeChinese('stt', word_to_idx), dtype = np.int32).reshape(1,-1)
test_data = np.concatenate([test_data] * 8, axis = 0)

state_ = sess.run(cell.zero_state(batch_size, tf.float32))

printer = ''
for i in xrange(500):
    predict, state_ = sess.run([logits, final_state], feed_dict = {input_data:test_data,\
                                                                   initial_state: state_})
    ans = np.argmax(predict, axis = 1)

    #ans = ans[:8]
    
    test_data = ans.reshape(-1,1)
    printer += cn_utils.decodeChinese(ans[0:1], idx_to_word)
print printer

去年今日此门中，人面桃花相映红。人面不知何处去，桃花依旧笑春风。end朋无一字，老病有孤舟。戎马关山北，凭轩涕泗流。end为在岐路，儿女共沾巾。end柴扉。春草年年绿，王孙归不归。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end入荷花去，佯羞不出来。end阵还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end水，萋萋满别情。end水，红掌拨清波。end


In [18]:
from IPython.display import clear_output, Image, display, HTML

# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(tf.get_default_graph())